# CGAN (conditional GAN)

**Author:** Teebone Ding

**Date:** 2019 Feb

Based on Conditional Generative Adversarial Nets (Mirza 2014) [Paper Link](https://arxiv.org/pdf/1411.1784.pdf)

Some useful refs while I implementing my cGAN:
* keras GAN [Link](https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py)
* cGAN in tensorflow version [Link](https://github.com/znxlwm/tensorflow-MNIST-cGAN-cDCGAN)
* Paper summary in Chinese version on Zhihu [Link](https://zhuanlan.zhihu.com/p/23648795)

Using MNIST dataset as playground

My SW/HW setting:
* ASUS nVidia GTX 1060 3GB
* RAM: 16GB
* CPU: Intel Core i5-7400 3.00GHz
* Ubuntu 16.04 LTS

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

1.12.0


## Load MNIST data and data explore

In [3]:
# Load MNIST data
mnist_dataset = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist_dataset.load_data()

In [4]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [5]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_ont_hot = to_categorical(y_test)

In [6]:
print(y_train_one_hot.shape)
print(y_train_one_hot[:10])

(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [7]:
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

# Preprocess training data

In [8]:
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
#x_train = x_train.astype(np.float32)/255.0

In [9]:
x_train = np.expand_dims(x_train,axis=3) # add color channel

In [10]:
x_train.shape

(60000, 28, 28, 1)

## Build Generator and Discriminator
In (Mirza 2014), the paper did not described its architecture really nice. I've modified several parts in both generator and discriminator. This architecture only contain fully connected (FC) layers. In my next iPython notebook, I will try CNN layers to build GAN (cDCGAN).

In [11]:
from tensorflow.keras.layers import Input, Dense, concatenate, Reshape, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD

In [12]:
z = Input(shape=(100,))    # Uniform distribution noise input
x = Input(shape=(28,28,1)) # image, by real dataset or generator
y = Input(shape=(10,))     # label, MNIST one hot encoded with 10 labels

In [13]:
def build_generator(z,y):
    #z_1 = Dense(200,activation='relu')(z)
    #y_1 = Dense(1000,activation='relu')(y)
    layer = concatenate([z,y])
    layer = Dense(128,activation='relu')(layer)
    layer = Dense(512,activation='relu')(layer)
    layer = Dense(28*28*1,activation='tanh')(layer)
    layer = Reshape((28,28,1))(layer)
    
    model = Model([z,y],layer)

    return model

G = build_generator(z,y)
G.summary() # not compiled, combined with discriminator and compile.

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 110)          0           input_1[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          14208       concatenate[0][0]                
__________

In [14]:
def build_discriminator(x,y):
    x_flatten = Flatten()(x)
    #x_1 = Dense(240,activation='relu')(x_flatten)
    #y_1 = Dense(50,activation='relu')(y)
    layer = concatenate([x_flatten,y])
    layer = Dense(240,activation='relu')(layer)
    layer = Dense(1,activation='sigmoid')(layer)
    
    model = Model([x,y],layer)
    return model

D = build_discriminator(x,y)

In [15]:
# While training....
# 1. train discriminator first
# 2. train a combined (G+D) model, with D is not trainable (only train G)
# 3. calculate loss value and save weights

In [16]:
# Compile Discriminator model
sgd = SGD(lr=0.1, momentum=0.5 ,decay= 1.00004)
adam = Adam(0.0002, 0.5)
D.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
D.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 784)          0           input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 794)          0           flatten[0][0]                    
                                                                 input_3[0][0]                    
__________

In [17]:
# Compile combined G+D model with D is not trainable
img = G([z,y])
D.trainable = False
valid = D([img,y])
C = Model([z,y], valid)
C.summary()
C.compile(optimizer=adam,loss='binary_crossentropy')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
model (Model)                   (None, 28, 28, 1)    482448      input_1[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            191041      model[1][0]                      
          

# Train conditional GAN

In [18]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)

In [19]:
# dataset: x_train, y_train_one_hot

In [20]:
# Randomly generate images from Generator network
def sample_gen_imgs(epoch):
    gen_labels = np.repeat(np.arange(10),10)
    gen_labels_cat = to_categorical(gen_labels, num_classes=10)
    noises = np.random.normal(0,1,(100,100))
    gen_imgs = G.predict([noises,gen_labels_cat])
    gen_imgs = 0.5 * gen_imgs + 0.5

    r,c = 10,10
    fig, axs = plt.subplots(r, c, figsize=(20, 20))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
            axs[i,j].set_title("Digit: %d" % gen_labels[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/gen_%d.png"%epoch)
    plt.close()

In [21]:
#import pdb
import datetime
def train(epochs = 10 ,batch_size = 128): 
    real_label = np.ones((batch_size,1))
    fake_label = np.zeros((batch_size,1))
    
    for epoch in range(epochs):
        epoch += 1
        for step in range(int(60000/batch_size)):
            # Train Discriminator once
            # sample real data from training dataset
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            imgs, labels = x_train[idx], y_train_one_hot[idx]
            # sample noises from normal dist.
            # Somehow I tried uniform dist but cannot trained very well.
            noises = np.random.normal(0,1,(batch_size,100))
            # generate fake images
            gen_imgs = G.predict([noises,labels])
            # Train Discriminator
            d_real_loss = D.train_on_batch([imgs,labels], real_label) 
            d_fake_loss = D.train_on_batch([gen_imgs,labels],fake_label)
            d_loss = 0.5*np.add(d_real_loss,d_fake_loss)
           
            # Train Combined model (generator)
            gen_labels = to_categorical(np.random.randint(0,10, batch_size),num_classes=10)
            # train generator
            g_loss = C.train_on_batch([noises,gen_labels], real_label)
            print("[%s] D-Loss value: %.5f Acc: %.5f G-Loss value: %.5f in epoch: %d"%(datetime.datetime.now(),d_loss[0],d_loss[1], g_loss, epoch),end='\r')
        
        if epoch % 10 == 0:
            print("[%s] D-Loss value: %.5f Acc: %.5f G-Loss value: %.5f in epoch: %d"%(datetime.datetime.now(),d_loss[0],d_loss[1], g_loss, epoch))
            sample_gen_imgs(epoch)
            G.save("models/G_%d.h5"%epoch)
            C.save("models/C_%d.h5"%epoch)
            D.save("models/D_%d.h5"%epoch)

In [22]:
train(epochs=200, batch_size = 100)

[2019-02-14 11:45:42.786548] D-Loss value: 0.26010 Acc: 0.94500 G-Loss value: 2.56396 in epoch: 10
[2019-02-14 11:46:32.619304] D-Loss value: 0.60281 Acc: 0.64500 G-Loss value: 1.37500 in epoch: 20
[2019-02-14 11:47:28.262280] D-Loss value: 0.70990 Acc: 0.56500 G-Loss value: 0.96813 in epoch: 30
[2019-02-14 11:48:21.934251] D-Loss value: 0.72252 Acc: 0.45500 G-Loss value: 0.80350 in epoch: 40
[2019-02-14 11:49:14.355797] D-Loss value: 0.71558 Acc: 0.50000 G-Loss value: 0.84901 in epoch: 50
[2019-02-14 11:50:04.852742] D-Loss value: 0.74334 Acc: 0.43000 G-Loss value: 0.81839 in epoch: 60
[2019-02-14 11:50:55.733737] D-Loss value: 0.68448 Acc: 0.58000 G-Loss value: 0.85500 in epoch: 70
[2019-02-14 11:51:47.507269] D-Loss value: 0.71111 Acc: 0.48500 G-Loss value: 0.80015 in epoch: 80
[2019-02-14 11:52:37.059630] D-Loss value: 0.69949 Acc: 0.51500 G-Loss value: 0.77844 in epoch: 90
[2019-02-14 11:53:25.365653] D-Loss value: 0.66602 Acc: 0.56000 G-Loss value: 0.83334 in epoch: 100
[2019-02-

## Generated images after 200 epochs
![alt text](images/gen_200.png "Gen image 200 epochs")
